In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    classification_report, 
    confusion_matrix, 
    roc_auc_score, 
    roc_curve, 
    accuracy_score
)

# Mengatur agar plot matplotlib tampil inline di notebook
%matplotlib inline
# Mengatur style plot seaborn
sns.set_style('whitegrid')

# 1. Memuat Data yang Telah Diproses

Memuat data `X_train_scaled`, `X_test_scaled`, `y_train`, dan `y_test` yang telah disimpan oleh Notebook `01_EDA_dan_Preprocessing.ipynb`.

In [ ]:
# Tentukan path
path_prefix = '../data/'

# Muat data
X_train_scaled = pd.read_csv(path_prefix + 'X_train_scaled.csv')
X_test_scaled = pd.read_csv(path_prefix + 'X_test_scaled.csv')
y_train = pd.read_csv(path_prefix + 'y_train.csv').values.ravel() # .values.ravel() untuk mengubah dari DataFrame ke 1D array
y_test = pd.read_csv(path_prefix + 'y_test.csv').values.ravel()

print("Data berhasil dimuat.")
print(f"Bentuk X_train_scaled: {X_train_scaled.shape}")
print(f"Bentuk y_train: {y_train.shape}")
print(f"Bentuk X_test_scaled: {X_test_scaled.shape}")
print(f"Bentuk y_test: {y_test.shape}")

# 2. Modeling: Logistic Regression

Melatih (training) model Logistic Regression menggunakan data latih yang sudah di-scaling.

In [ ]:
# Inisialisasi model
# random_state=42 agar hasil training konsisten dan bisa direproduksi
model_lr = LogisticRegression(random_state=42)

# Latih (fit) model pada data training
model_lr.fit(X_train_scaled, y_train)

print("Model Logistic Regression berhasil dilatih.")

# 3. Evaluasi Model

Mengevaluasi performa model pada data uji (test set) menggunakan semua metrik yang telah direncanakan di proposal (Bab 3.3).

In [ ]:
# Gunakan model yang sudah dilatih untuk memprediksi data test
y_pred = model_lr.predict(X_test_scaled)

In [ ]:
# Menampilkan metrik utama: Precision, Recall, F1-Score
print("Laporan Klasifikasi (Classification Report):")
print(classification_report(y_test, y_pred, target_names=['Selamat (0)', 'Meninggal (1)']))

# Catatan: Perhatikan skor Recall untuk kelas 'Meninggal (1)'.
# Ini adalah metrik penting dalam kasus medis (kemampuan model menemukan pasien yang berisiko).

In [ ]:
# Hitung prediksi probabilitas untuk kelas 1 (Meninggal)
y_prob = model_lr.predict_proba(X_test_scaled)[:, 1]

# Hitung metrik
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_prob)

print(f"Akurasi (Accuracy): {accuracy * 100:.2f}%")
print(f"ROC-AUC Score: {roc_auc:.4f}")

# 4. Visualisasi Hasil Evaluasi

Membuat visualisasi yang direncanakan di proposal (Bab 3.4) untuk menganalisis performa model.

In [ ]:
# Membuat confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Visualisasi dengan heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Prediksi Selamat (0)', 'Prediksi Meninggal (1)'],
            yticklabels=['Aktual Selamat (0)', 'Aktual Meninggal (1)'])
plt.title('Confusion Matrix')
plt.ylabel('Label Aktual')
plt.xlabel('Label Prediksi')
plt.show()

In [ ]:
# Hitung FPR (False Positive Rate) dan TPR (True Positive Rate)
fpr, tpr, thresholds = roc_curve(y_test, y_prob)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC Curve (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], color='red', lw=2, linestyle='--', label='Garis Acak (AUC = 0.5)')
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()

# 5. Analisis Koefisien (Feature Importance)

Menganalisis koefisien dari model Logistic Regression untuk memahami fitur mana yang paling berpengaruh dalam memprediksi mortalitas, sesuai tujuan proposal (Bab 1.3 dan Bab 3.4).

In [ ]:
# Ambil koefisien dari model
coefficients = model_lr.coef_[0]

# Ambil nama fitur dari data X_train
feature_names = X_train_scaled.columns

# Buat DataFrame untuk memudahkan visualisasi
coef_df = pd.DataFrame({'Fitur': feature_names, 'Koefisien': coefficients})

# Urutkan berdasarkan nilai absolut koefisien untuk melihat pengaruh terkuat
coef_df['Koefisien Absolut'] = coef_df['Koefisien'].abs()
coef_df = coef_df.sort_values(by='Koefisien', ascending=False)

display(coef_df)

In [ ]:
plt.figure(figsize=(10, 8))
sns.barplot(x='Koefisien', y='Fitur', data=coef_df, palette='vlag')
plt.title('Peringkat Kepentingan Fitur (Berdasarkan Koefisien Logistic Regression)')
plt.xlabel('Nilai Koefisien (Pengaruh terhadap Prediksi "Meninggal")')
plt.ylabel('Fitur')
plt.show()

print("""
Interpretasi Koefisien:
- Batang Positif (Biru): Fitur yang berkontribusi terhadap prediksi 'Meninggal' (DEATH_EVENT = 1).
  Semakin panjang batangnya, semakin kuat pengaruhnya.
- Batang Negatif (Merah): Fitur yang berkontribusi terhadap prediksi 'Selamat' (DEATH_EVENT = 0).
  Semakin panjang batangnya ke kiri, semakin kuat pengaruh 'protektif'-nya.
""")

# Selesai

Modeling, evaluasi, dan analisis telah selesai. Hasil ini siap untuk disusun menjadi laporan akhir (Bab 4.2).